<a href="https://colab.research.google.com/github/marciogsantana/Ciencia-de-dados/blob/main/transporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#programa utilizado para o exemplo de grafos em transporte

In [ ]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [ ]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("transporteGrafos").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [ ]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv,Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv,463094961
dbfs:/FileStore/tables/d1995_07_01-24d0c.json,d1995_07_01-24d0c.json,7451741
dbfs:/FileStore/tables/d1995_07_02-c3f44.json,d1995_07_02-c3f44.json,9572086
dbfs:/FileStore/tables/d1995_07_03-f99af.json,d1995_07_03-f99af.json,14125303
dbfs:/FileStore/tables/d1995_07_04-c5a7f.json,d1995_07_04-c5a7f.json,12030333
dbfs:/FileStore/tables/d1995_07_05-0f261.json,d1995_07_05-0f261.json,14662194
dbfs:/FileStore/tables/d1995_07_06-20619.json,d1995_07_06-20619.json,15557682
dbfs:/FileStore/tables/d1995_07_07-2dd8d.json,d1995_07_07-2dd8d.json,15279295
dbfs:/FileStore/tables/d1995_07_08-83302.json,d1995_07_08-83302.json,7033852
dbfs:/FileStore/tables/d1995_07_09-f75d3.json,d1995_07_09-f75d3.json,5589076


Carregando o arquivo

In [ ]:
nosTransporte="/FileStore/tables/transport_nodes-7c826.csv"  #diretório que contém o arquivo a ser utilizado para os nos

In [ ]:
relacionamentosTransporte="/FileStore/tables/transport_relationships-c2bfc.csv"  #diretório que contém o arquivo a ser utilizado

In [ ]:
#importando os banco de dados
dfNos = spark.read.format('csv').options(header='true',delimiter=',', inferSchema=True).load(nosTransporte)

In [ ]:
dfNos.show()

+----------------+---------+---------+----------+
 id| latitude|longitude|population|
+----------------+---------+---------+----------+
 Amsterdam|52.379189| 4.899431| 821752|
 Utrecht|52.092876| 5.10448| 334176|
 Den Haag|52.078663| 4.288788| 514861|
 Immingham| 53.61239| -0.22219| 9642|
 Doncaster| 53.52285| -1.13116| 302400|
Hoek van Holland| 51.9775| 4.13333| 9382|
 Felixstowe| 51.96375| 1.3511| 23689|
 Ipswich| 52.05917| 1.15545| 133384|
 Colchester| 51.88921| 0.90421| 104390|
 London|51.509865|-0.118092| 8787892|
 Rotterdam| 51.9225| 4.47917| 623652|
 Gouda| 52.01667| 4.70833| 70939|
+----------------+---------+---------+----------+

In [ ]:
dfRelacionamentos = spark.read.format('csv').options(header='true',delimiter=',', inferSchema=True).load(relacionamentosTransporte)

In [ ]:
dfRelacionamentos.show()

+----------------+----------------+------------+----+
 src| dst|relationship|cost|
+----------------+----------------+------------+----+
 Amsterdam| Utrecht| EROAD| 46|
 Amsterdam| Den Haag| EROAD| 59|
 Den Haag| Rotterdam| EROAD| 26|
 Amsterdam| Immingham| EROAD| 369|
 Immingham| Doncaster| EROAD| 74|
 Doncaster| London| EROAD| 277|
Hoek van Holland| Den Haag| EROAD| 27|
 Felixstowe|Hoek van Holland| EROAD| 207|
 Ipswich| Felixstowe| EROAD| 22|
 Colchester| Ipswich| EROAD| 32|
 London| Colchester| EROAD| 106|
 Gouda| Rotterdam| EROAD| 25|
 Gouda| Utrecht| EROAD| 35|
 Den Haag| Gouda| EROAD| 32|
Hoek van Holland| Rotterdam| EROAD| 33|
+----------------+----------------+------------+----+

Definindo e criando o grafo

In [ ]:
#importando as funções para utilizar os grafos
from pyspark.sql.types import *
from graphframes import *  #contém os métodos para serem utilizados no processamento através dos grafos
#obs: a biblioteca graphframes deve ser adicionada ao databricks, pois não é nativa. Para isso acesse New->"Library"-> "Source"-> "Maven Coordinate"-> pesquise por "graphframes" e adicione a biblioteca para cada cluster

In [ ]:
#definindo o "Esquema" para cada um dos nós
atributosNo = [
StructField("id", StringType(), True),\
StructField("latitude", FloatType(), True),\
StructField("longitude", FloatType(), True),\
StructField("population", IntegerType(), True)\
]

In [ ]:
#importa os dados como nós
nos = spark.read.csv(nosTransporte, header=True,schema=StructType(atributosNo))

Definindo os relacionamentos diretos

In [ ]:
#importa os dados como relacionamentos
relaci_direto = spark.read.csv(relacionamentosTransporte, header=True)

In [ ]:
relaci_direto.show(5)

+---------+---------+------------+----+
 src| dst|relationship|cost|
+---------+---------+------------+----+
Amsterdam| Utrecht| EROAD| 46|
Amsterdam| Den Haag| EROAD| 59|
 Den Haag|Rotterdam| EROAD| 26|
Amsterdam|Immingham| EROAD| 369|
Immingham|Doncaster| EROAD| 74|
+---------+---------+------------+----+
only showing top 5 rows

Definindo os relacionamentos inversos

In [ ]:
#define os relacionamentos com a troca das fontes e destinos
relaci_inverso = (relaci_direto.withColumn("newSrc", relaci_direto.dst).withColumn("newDst", relaci_direto.src)\
.drop("dst", "src")\
.withColumnRenamed("newSrc", "src")\
.withColumnRenamed("newDst", "dst")\
.select("src", "dst", "relationship", "cost"))\

In [ ]:
relaci_inverso.show(5)

+---------+---------+------------+----+
 src| dst|relationship|cost|
+---------+---------+------------+----+
 Utrecht|Amsterdam| EROAD| 46|
 Den Haag|Amsterdam| EROAD| 59|
Rotterdam| Den Haag| EROAD| 26|
Immingham|Amsterdam| EROAD| 369|
Doncaster|Immingham| EROAD| 74|
+---------+---------+------------+----+
only showing top 5 rows

In [ ]:
#cria o df com os relacionamentos diretos e inverso (torna o grafo bidirecional)
relacionamentos=relaci_direto.union(relaci_inverso)

In [ ]:
relacionamentos.show()

+----------------+----------------+------------+----+
 src| dst|relationship|cost|
+----------------+----------------+------------+----+
 Amsterdam| Utrecht| EROAD| 46|
 Amsterdam| Den Haag| EROAD| 59|
 Den Haag| Rotterdam| EROAD| 26|
 Amsterdam| Immingham| EROAD| 369|
 Immingham| Doncaster| EROAD| 74|
 Doncaster| London| EROAD| 277|
Hoek van Holland| Den Haag| EROAD| 27|
 Felixstowe|Hoek van Holland| EROAD| 207|
 Ipswich| Felixstowe| EROAD| 22|
 Colchester| Ipswich| EROAD| 32|
 London| Colchester| EROAD| 106|
 Gouda| Rotterdam| EROAD| 25|
 Gouda| Utrecht| EROAD| 35|
 Den Haag| Gouda| EROAD| 32|
Hoek van Holland| Rotterdam| EROAD| 33|
 Utrecht| Amsterdam| EROAD| 46|
 Den Haag| Amsterdam| EROAD| 59|
 Rotterdam| Den Haag| EROAD| 26|
 Immingham| Amsterdam| EROAD| 369|
 Doncaster| Immingham| EROAD| 74|
+----------------+----------------+------------+----+
only showing top 20 rows

In [ ]:
#define o grafo 
grafo=GraphFrame(nos, relacionamentos)

Explorando o nosso grafo

In [ ]:
#encontrando quais são as cidades com mais de 100000 habitantes e menos de 300000
popMedia=grafo.vertices\
.filter("population > 100000 and population < 300000")\
.sort("population")\
.show()

+----------+--------+---------+----------+
 id|latitude|longitude|population|
+----------+--------+---------+----------+
Colchester|51.88921| 0.90421| 104390|
 Ipswich|52.05917| 1.15545| 133384|
+----------+--------+---------+----------+

In [ ]:
#mostrando a quantidade de caminhos diretos (chegando)
display(grafo.inDegrees)

id,inDegree
Doncaster,2
Rotterdam,3
London,2
Den Haag,4
Immingham,2
Amsterdam,3
Colchester,2
Utrecht,2
Gouda,3
Ipswich,2


In [ ]:
#mostrando a quantidade de caminhos inversos (saindo)
display(grafo.outDegrees)

id,outDegree
Doncaster,2
Rotterdam,3
London,2
Den Haag,4
Immingham,2
Amsterdam,3
Colchester,2
Utrecht,2
Gouda,3
Ipswich,2


In [ ]:
#qual é o nó mais "importante" (tem mais caminhos que levam até ele)
total_degree = grafo.degrees
in_degree = grafo.inDegrees
out_degree = grafo.outDegrees


In [ ]:
total_degree.show()

+----------------+------+
 id|degree|
+----------------+------+
 Doncaster| 4|
 Rotterdam| 6|
 London| 4|
 Den Haag| 8|
 Immingham| 4|
 Amsterdam| 6|
 Colchester| 4|
 Utrecht| 4|
 Gouda| 6|
 Ipswich| 4|
Hoek van Holland| 6|
 Felixstowe| 4|
+----------------+------+

In [ ]:
total_degree.join(in_degree, "id", how="left")\
.join(out_degree, "id", how="left")\
.fillna(0)\
.sort("inDegree", ascending=False)\
.show()

+----------------+------+--------+---------+
 id|degree|inDegree|outDegree|
+----------------+------+--------+---------+
 Den Haag| 8| 4| 4|
 Gouda| 6| 3| 3|
 Rotterdam| 6| 3| 3|
Hoek van Holland| 6| 3| 3|
 Amsterdam| 6| 3| 3|
 Doncaster| 4| 2| 2|
 Immingham| 4| 2| 2|
 London| 4| 2| 2|
 Utrecht| 4| 2| 2|
 Ipswich| 4| 2| 2|
 Felixstowe| 4| 2| 2|
 Colchester| 4| 2| 2|
+----------------+------+--------+---------+

In [ ]:
#realizando consultas
motifs = grafo.find("(Amsterdam)-[e]->(Utrecht)")
display(motifs)

Amsterdam,e,Utrecht
"List(Amsterdam, 52.37919, 4.899431, 821752)","List(Amsterdam, Utrecht, EROAD, 46)","List(Utrecht, 52.092876, 5.10448, 334176)"
"List(Amsterdam, 52.37919, 4.899431, 821752)","List(Amsterdam, Den Haag, EROAD, 59)","List(Den Haag, 52.078663, 4.288788, 514861)"
"List(Den Haag, 52.078663, 4.288788, 514861)","List(Den Haag, Rotterdam, EROAD, 26)","List(Rotterdam, 51.9225, 4.47917, 623652)"
"List(Amsterdam, 52.37919, 4.899431, 821752)","List(Amsterdam, Immingham, EROAD, 369)","List(Immingham, 53.61239, -0.22219, 9642)"
"List(Immingham, 53.61239, -0.22219, 9642)","List(Immingham, Doncaster, EROAD, 74)","List(Doncaster, 53.52285, -1.13116, 302400)"
"List(Doncaster, 53.52285, -1.13116, 302400)","List(Doncaster, London, EROAD, 277)","List(London, 51.509865, -0.118092, 8787892)"
"List(Hoek van Holland, 51.9775, 4.13333, 9382)","List(Hoek van Holland, Den Haag, EROAD, 27)","List(Den Haag, 52.078663, 4.288788, 514861)"
"List(Felixstowe, 51.96375, 1.3511, 23689)","List(Felixstowe, Hoek van Holland, EROAD, 207)","List(Hoek van Holland, 51.9775, 4.13333, 9382)"
"List(Ipswich, 52.05917, 1.15545, 133384)","List(Ipswich, Felixstowe, EROAD, 22)","List(Felixstowe, 51.96375, 1.3511, 23689)"
"List(Colchester, 51.88921, 0.90421, 104390)","List(Colchester, Ipswich, EROAD, 32)","List(Ipswich, 52.05917, 1.15545, 133384)"


In [ ]:
#filtrando os resultados da consulta anterior
filtered = motifs.filter("e.cost < 30")
display(filtered)

Amsterdam,e,Utrecht
"List(Den Haag, 52.078663, 4.288788, 514861)","List(Den Haag, Rotterdam, EROAD, 26)","List(Rotterdam, 51.9225, 4.47917, 623652)"
"List(Hoek van Holland, 51.9775, 4.13333, 9382)","List(Hoek van Holland, Den Haag, EROAD, 27)","List(Den Haag, 52.078663, 4.288788, 514861)"
"List(Ipswich, 52.05917, 1.15545, 133384)","List(Ipswich, Felixstowe, EROAD, 22)","List(Felixstowe, 51.96375, 1.3511, 23689)"
"List(Gouda, 52.01667, 4.70833, 70939)","List(Gouda, Rotterdam, EROAD, 25)","List(Rotterdam, 51.9225, 4.47917, 623652)"
"List(Rotterdam, 51.9225, 4.47917, 623652)","List(Rotterdam, Den Haag, EROAD, 26)","List(Den Haag, 52.078663, 4.288788, 514861)"
"List(Den Haag, 52.078663, 4.288788, 514861)","List(Den Haag, Hoek van Holland, EROAD, 27)","List(Hoek van Holland, 51.9775, 4.13333, 9382)"
"List(Felixstowe, 51.96375, 1.3511, 23689)","List(Felixstowe, Ipswich, EROAD, 22)","List(Ipswich, 52.05917, 1.15545, 133384)"
"List(Rotterdam, 51.9225, 4.47917, 623652)","List(Rotterdam, Gouda, EROAD, 25)","List(Gouda, 52.01667, 4.70833, 70939)"


In [ ]:
#encontrando o menor caminho entre a cidade de Den Haag e alguma das cidades com tamanho médio
origem = "id='Den Haag'"
destino = "population > 100000 and population < 300000 and id <> 'Den Haag'"
resultado = grafo.bfs(origem, destino) #bfs encontra o menor caminho entre dois nós

In [ ]:
print(resultado.columns)  #colunas com 'e' representam as arestas (edges) e colunas com 'v' representam os nós (vértices)

['from', 'e0', 'v1', 'e1', 'v2', 'e2', 'to']

In [ ]:
#selecionamdo apenas os nós (selecionando quem não começa com e)
colunas = [coluna for coluna in resultado.columns if not coluna.startswith("e")]
resultado.select(colunas).show(5,False)

+---------------------------------------+------------------------------------------+-------------------------------------+------------------------------------+
from |v1 |v2 |to |
+---------------------------------------+------------------------------------------+-------------------------------------+------------------------------------+
[Den Haag, 52.078663, 4.288788, 514861]|[Hoek van Holland, 51.9775, 4.13333, 9382]|[Felixstowe, 51.96375, 1.3511, 23689]|[Ipswich, 52.05917, 1.15545, 133384]|
+---------------------------------------+------------------------------------------+-------------------------------------+------------------------------------+

In [ ]:
grafo.vertices\
.filter("population > 100000 and population < 300000")\
.sort("population")\
.show()

+----------+--------+---------+----------+
 id|latitude|longitude|population|
+----------+--------+---------+----------+
Colchester|51.88921| 0.90421| 104390|
 Ipswich|52.05917| 1.15545| 133384|
+----------+--------+---------+----------+